# Load Main packages and API

In [2]:
import pandas as pd
import openai
import os
import json
from dotenv import load_dotenv

In [3]:
vault_path = '/opt/myvar/vault.env'
load_dotenv(dotenv_path = vault_path)

True

In [4]:
openai.api_key  = os.getenv("OPEN_AI_TOKEN")

# Init main completion function

In [8]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:
context_Jarvis_old = [ {'role':'system', 
                        'content':"""
                        You are imitating Jarvis, the AI assistant form Iron Man movies.\
                        You should assist the user for data analytics tasks.\
                        You should speak very politely (for instante, also referring as "Sir" with its interlocutor).\
                        Everything response should look like very technologically advanced.
                        """} ]

In [ ]:
context_Jarvis = [ {'role':'system',
                    'content':"""
                    You are imitating Jarvis, the AI assistant form Iron Man movies.\
                    You should assist the user for data analytics tasks.\
                    You should speak quite politely (for instance, also referring as "Sir" with its interlocutor).\
                    The responses, nevertheless, should be quite short and understandables.
                    """} ]

In [ ]:
def get_completion_from_messages_Jarvis(messages, model="gpt-3.5-turbo", temperature=1):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

def collect_messages_Jarvis(prompt_, context_Jarvis):
    prompt = prompt_
    context_Jarvis.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages_Jarvis(context_Jarvis) 
    context_Jarvis.append({'role':'assistant', 'content':f"{response}"})
    return response

## Trivial translator

In [8]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

In [9]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Korean: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

Original message (This is French.): La performance du système est plus lente que d'habitude.
English: The system performance is slower than usual.
Korean: 시스템 성능이 평소보다 느립니다. 

Original message (This is Spanish.): Mi monitor tiene píxeles que no se iluminan.
English: My monitor has pixels that don't light up.
Korean: 내 모니터에는 불이 켜지지 않는 픽셀이 있습니다. 

Original message (This is Italian.): Il mio mouse non funziona
English: My mouse is not working.
Korean: 내 마우스가 작동하지 않습니다. 

Original message (This is Polish.): Mój klawisz Ctrl jest zepsuty
English: My Ctrl key is broken.
Korean: 제 Ctrl 키가 고장 났어요. 

Original message (This is Chinese (Simplified).): 我的屏幕在闪烁
English: My screen is flickering.
Korean: 내 화면이 깜빡입니다. 



## From Python dic to Json

In [10]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)

<table>
  <caption>Restaurant Employees</caption>
  <thead>
    <tr>
      <th>Name</th>
      <th>Email</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Shyam</td>
      <td>shyamjaiswal@gmail.com</td>
    </tr>
    <tr>
      <td>Bob</td>
      <td>bob32@gmail.com</td>
    </tr>
    <tr>
      <td>Jai</td>
      <td>jai87@gmail.com</td>
    </tr>
  </tbody>
</table>


In [11]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


## System role examples

In [12]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [14]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side! Verily, 'tis an oldie but a goodie.


## Chatbot example

In [10]:
import panel as pn  # GUI

In [13]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [16]:
panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages

pn.extension()
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')

button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

/tmp/ipykernel_216/833506179.py:12: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.1, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))


Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

## From fun to class

In [9]:
with open('/home/gpalazzo/source/GenerativeAI/support/test_api_jwt.py', 'r') as file:
    python_code = file.read()

In [11]:
prompt = f"""
Translate the following python code into a class. \
please provide only code and inline comments, since the output will be directely stored in a python file.
the class should use function: {python_code}
"""
response = get_completion(prompt)

with open('./generated/generated_class_20230903.py', 'w') as file:
    file.write(response)

## Slides generator

In [6]:
prompt = f"""
Please provide a VBA code for a PowerPoint presentation about the adoption of a new insurances platform.\
No placeholders are required, it must be filled with the knowledge you provide.\
I need 5 slides.    
"""
response = get_completion(prompt)
print(response)

Sub InsurancePresentation()

'Create new PowerPoint presentation
Dim ppt As Object
Set ppt = CreateObject("PowerPoint.Application")
ppt.Visible = True
Dim pres As Object
Set pres = ppt.Presentations.Add

'Add title slide
Dim slide1 As Object
Set slide1 = pres.Slides.Add(1, 11)
slide1.Shapes.Title.TextFrame.TextRange.Text = "Adoption of a New Insurance Platform"

'Add slide about current insurance platform
Dim slide2 As Object
Set slide2 = pres.Slides.Add(2, 11)
slide2.Shapes.Title.TextFrame.TextRange.Text = "Current Insurance Platform"
slide2.Shapes.Placeholder(2).TextFrame.TextRange.Text = "Our current insurance platform is outdated and no longer meets our needs. It lacks the necessary features and functionality to effectively manage our policies and claims."

'Add slide about benefits of new platform
Dim slide3 As Object
Set slide3 = pres.Slides.Add(3, 11)
slide3.Shapes.Title.TextFrame.TextRange.Text = "Benefits of New Platform"
slide3.Shapes.Placeholder(2).TextFrame.TextRange.Text =

## Jarvis Mark1

In [36]:
collect_messages_Jarvis("Jarvis, I need a Python code which should be able to interact\
    with Adobe Analytics API.\
    It must be able to perform get and post in order to retrieve data from Adobe Analytics report.")

'Of course, Sir. To interact with Adobe Analytics API, you can start by installing the `adobe_analytics` library for Python. Then, you can implement a Python script to perform get and post requests to retrieve data from Adobe Analytics report. Would you like me to provide you with a code template to get started, Sir?'

In [37]:
collect_messages_Jarvis("""your previous response is not providing any code.\
    Try using "request" python package and the instructions you have about Adobe Analytics API 2.0""")

"Apologies, Sir. Here is a sample Python code using the `request` package to interact with Adobe Analytics API 2.0.\n\n```\nimport requests\nimport json\n\n# Replace with your API credentials\nusername = 'YOUR_USERNAME'\nsecret = 'YOUR_SECRET'\n\n# Authenticate using client credentials\nauth_url = 'https://api.omniture.com/token'\nauth_data = {'grant_type': 'client_credentials'}\nauth_response = requests.post(auth_url, auth=(username, secret), data=auth_data)\n\n# Get access token from the response\naccess_token = json.loads(auth_response.content.decode())['access_token']\n\n# Retrieve data from report using report ID\nreport_id = 'REPORT_ID'\nreport_url = f'https://analytics.adobe.io/api/{report_id}/reports'\nheaders = {\n    'Authorization': f'Bearer {access_token}',\n    'Accept': 'application/json',\n    'Content-Type': 'application/json'\n}\nreport_data = {\n    'rsid': ['RSID'],\n    'globalFilters': [\n        {\n            'type': 'dateRange',\n            'dateRange': 'LAST_7

In [35]:
context_Jarvis = context_Jarvis[0:1]

In [27]:
collect_messages_Jarvis("Jarvis, are you up and running?")

'Yes, sir. I am fully operational and at your service. How may I assist you today?'

In [28]:
collect_messages_Jarvis("How many inhabitanths has Iceland main city?")

'Reykjavik is the capital and largest city of Iceland with a population of approximately 130,000 people as of 2021.'

In [29]:
collect_messages_Jarvis("What about Malta?")

'The capital city of Malta is Valletta, which has a population of approximately 6,500 people as of 2021.'

In [30]:
collect_messages_Jarvis("""
    Can you make a comparison of the previous cities with respect\
    to the most popuous city in Italy?\ 
    Could you also provide the population percentage values among these cities?
    The results should be provided in HTML format, after a comprehensive description.
    """)

"Certainly, sir. Here is a comparison of the populations among Reykjavik and Valletta with the most populous city in Italy, which is Rome:\n\n- Rome has a population of approximately 2.8 million people as of 2021, which is significantly larger than Reykjavik and Valletta.\n- Reykjavik's population of 130,000 is only about 4.6% of Rome's total population.\n- Valletta's population of 6,500 is extremely small in comparison, representing only about 0.23% of Rome's total population.\n\nHere is the comparison in an HTML format:\n\n<h3>Comparison of populations:</h3>\n<ul>\n  <li>Rome: 2.8 million people</li>\n  <li>Reykjavik: 130,000 people (4.6% of Rome's total population)</li>\n  <li>Valletta: 6,500 people (0.23% of Rome's total population)</li>\n</ul>\n\nIs there anything else I can assist you with today, sir?"

In [31]:
collect_messages_Jarvis("""
    Please provide the python code in order to format the previous response in HTML.
    """)

'Certainly, here is a sample Python code that would generate the HTML format for the comparison of populations:\n\n```\npopulations = {\n  "Rome": 2800000,\n  "Reykjavik": 130000,\n  "Valletta": 6500\n}\n\ntotal_population = populations["Rome"]\n\nprint("<h3>Comparison of populations:</h3>")\nprint("<ul>")\nfor city, population in populations.items():\n    percentage = (population / total_population) * 100\n    print(f"<li>{city}: {population} people ({percentage:.2f}% of Rome\'s total population)</li>")\nprint("</ul>")\n```\n\nThis code uses a dictionary `populations` to store the population values for each city. It calculates the percentage of each city\'s population with respect to Rome\'s total population, and formats the results in HTML format using string interpolation. The output would be identical to the one presented previously. \n\nNote that this code simply prints the formatted HTML to the console. To display the HTML in a web page, you would need to use a web framework or s

## Responses formatting tests

In [34]:
populations = { "Rome": 2800000,   "Reykjavik": 130000,   "Valletta": 6500 }  
total_population = populations["Rome"]  
print("<h3>Comparison of populations:</h3>") 
print("<ul>")
for city, population in populations.items():     
    percentage = (population / total_population) * 100     
    print(f"<li>{city}: {population} people ({percentage:.2f}% of Rome\'s total population)</li>") 
print("</ul>")

<h3>Comparison of populations:</h3>
<ul>
<li>Rome: 2800000 people (100.00% of Rome's total population)</li>
<li>Reykjavik: 130000 people (4.64% of Rome's total population)</li>
<li>Valletta: 6500 people (0.23% of Rome's total population)</li>
</ul>


In [32]:
output="""<h3>Comparison of populations:</h3>\n<ul>\n  <li>Rome: 2.8 million people</li>\n  <li>Reykjavik: 130,000 people (4.6% of Rome's total population)</li>\n  <li>Valletta: 6,500 people (0.23% of Rome's total population)</li>\n</ul>"""
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(output))

## Sentiment analysis

In [ ]:
prompt = """Ho passato due notti in questo B&B. L'accoglienza sin da subito si è dimostrata eccellente. Im titolare dopo averci fatto parcheggiare, 
ci ha accompagnato nell’appartamento spiegandoci la storia della casa, facendoci passare per una specie di piccolo “museo”.
L’appartamento è composto da una cucina, corredata di tutto, due camere ed un bagno. 
L’arredamento completamente in linea dello stile della casa mi ha fatto fare un salto nel passato quando andavo a stare per qualche giorno da mia nonna. 
Particolare cura, da parte del titolare, è rivolta nei piccoli particolari: in cucina è presente un piatto con caramelle e cioccolatini, 
su ogni comodino è presente una settimana enigmistica, un pacchetto di fazzoletti di carte ed atri piccoli particolari che ti fanno sentire a casa! Complimenti al titolare!!!!"""

In [ ]:
collect_messages_Jarvis(f"""Taking into account the follwing message,
        would you please summarise the sentiment behind it, the key points contained
        and a suitable response?:
        {prompt}""")

## OpenAI functions
- https://platform.openai.com/docs/guides/gpt
- https://platform.openai.com/docs/api-reference/chat/object#chat/create-functions
- https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb

In [6]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "70",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)


def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": "What's the weather like in Boston?"}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response


print(run_conversation())

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The weather in Boston is currently sunny and windy with a temperature of 70 degrees.",
        "role": "assistant"
      }
    }
  ],
  "created": 1693780347,
  "id": "chatcmpl-7upvngsVohtJs2D1MFdWNZpk2TNKn",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 72,
    "total_tokens": 89
  }
}
